# Sources
* BiLSTM model architecture based on [Ozols et. al., 2021](https://www.mdpi.com/1422-0067/22/6/3071/htm)


In [ ]:
import os
import sys
import csv
import math
import random
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchtext.vocab import build_vocab_from_iterator

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [ ]:
def read_data(path):
    with open(path, "r") as csvfile:
        train_data = list(csv.reader(csvfile))[1:]  # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls


def apply_random_masking(seq, num_tokens):
    """
    Mask `num_tokens` as 0 at random positions per sequence.
    """
    dist = torch.rand(seq.shape)
    m, _ = torch.topk(dist, num_tokens)
    return seq * (dist < m[0, -1])


def regularized_auc(train_auc, dev_auc, threshold=0.0025):
    """
    Returns development AUC if overfitting is below threshold, otherwise 0.
    """
    return dev_auc if (train_auc - dev_auc) < threshold else 0


def save_metrics(*args, path):
    if not os.path.isfile(path):
        with open(path, "w", newline="\n") as f:
            f.write(
                ",".join(
                    [
                        "windowsize",
                        "fold",
                        "epoch",
                        "train_loss",
                        "train_acc",
                        "train_auc",
                        "val_loss",
                        "val_acc",
                        "val_auc",
                    ]
                )
            )
            f.write("\n")
    if args:
        with open(path, "a", newline="\n") as f:
            f.write(",".join([str(arg) for arg in args]))
            f.write("\n")


def gelu(x):
    """
    Facebook Research implementation of the gelu activation function.
    
    For information: OpenAI GPT's gelu is slightly different
    (and gives slightly different results):
    0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))    
            
def trainable_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def total_model_params(model):
    return sum(p.numel() for p in model.parameters())

In [ ]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl
    
    def __getitem__(self, idx):
        return self.seq[idx], self.lbl[idx]
    
    def __len__(self):
        return len(self.lbl)
    
class TrainBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.int64)
        num_masked = seq.shape[-1] // 10 if seq.shape[-1] // 10 > 0 else 1
        self.seq = apply_random_masking(seq, num_tokens=num_masked)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def train_wrapper(batch):
    return TrainBatch(batch)


class EvalBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        self.seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.int64)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def eval_wrapper(batch):
    return EvalBatch(batch)

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, rnn_size1, rnn_size2, hidden_size, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
        )
        
        self.dropout=nn.Dropout(dropout)
        
        self.lstm1 = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=rnn_size1,
            bidirectional=True,
            batch_first=True,
        )

        self.lstm2 = nn.LSTM(
            input_size=2 * rnn_size1,
            hidden_size=rnn_size2,
            bidirectional=True,
            batch_first=True,
        )
        
        self.fc1 = nn.Linear(rnn_size2 * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, seq):
        # input shape: (batch_size, seq_len=10)
        embedded = self.dropout(self.embedding(seq))
        
        # input shape: (batch_size, seq_len, embedding_dim)
        out, _ = self.lstm1(embedded)
        
        # input shape: (batch_size, seq_len, 2*rnn_size1)
        out, _ = self.lstm2(out)
        
        # input shape: (batch_size, seq_len, 2*hidden_size)
        pooled, _ = torch.max(out, dim=1)
        
        # input shape: (batch_size, 2*hidden_size)
        out = self.dropout(gelu(self.fc1(pooled)))
        
        # input shape: (batch_size, hidden_size)
        # output shape: (batch_size)
        return self.fc2(out).squeeze()

In [ ]:
def process(model, loader, criterion, optim=None):
    epoch_loss, num_correct, total = 0, 0, 0
    preds, lbls = [], []
    
    for batch in tqdm(
        loader,
        desc="Train: " if optim is not None else "Eval: ",
        file=sys.stdout,
        unit="batches"
    ):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores = model(seq)
        loss = criterion(scores, lbl)
        
        if optim is not None:
            optim.zero_grad()
            loss.backward()
            optim.step()
        
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum().item()
        total += len(seq)
        preds.extend(scores.detach().tolist())
        lbls.extend(lbl.detach().tolist())
    return epoch_loss / total, num_correct / total, roc_auc_score(lbls, preds)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 512

vocab = torch.load('../../data/vocab.pt')
vocab.set_default_index(vocab['<UNK>'])
encode_text = lambda x: vocab(list(x))

In [ ]:
NUM_EPOCHS = 15
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 76
RNN_SIZE1 = 252
RNN_SIZE2 = 518
HIDDEN_SIZE = 179
DROPOUT = 0.5
LEARNING_RATE = 3e-4

params = {
    "vocab_size": VOCAB_SIZE,
    "embedding_dim": EMBEDDING_DIM,
    "rnn_size1": RNN_SIZE1,
    "rnn_size2": RNN_SIZE2,
    "hidden_size": HIDDEN_SIZE,
    "dropout": DROPOUT
}

criterion = nn.BCEWithLogitsLoss()

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=1234)
windowsizes = ['3', '7', '14', '21', '28']
path = "../../params/n_term/windowsize_variation/"
logging_path = path + 'results.csv'

print("Starting Cross-Validation.")
start = time()
for windowsize in windowsizes:
    df = pd.read_csv('../../data/n_halfwindow' + windowsize + '.csv')
    
    highest_val_auc = 0
    # get a new fold
    for fold, (train_idx, dev_idx) in enumerate(kf.split(df), 1):
        X_tr = df.loc[train_idx, 'cleavage_window'].values
        y_tr = df.loc[train_idx, 'elution_cleavage'].values
        X_dev = df.loc[dev_idx, 'cleavage_window'].values
        y_dev = df.loc[dev_idx, 'elution_cleavage'].values

        # create datasets and loads with current split
        train_data = CleavageDataset(X_tr, y_tr)
        train_loader = DataLoader(
            train_data,
            batch_size=BATCH_SIZE,
            shuffle=True,
            collate_fn=train_wrapper,
            pin_memory=True,
            num_workers=10,
        )

        dev_data = CleavageDataset(X_dev, y_dev)
        dev_loader = DataLoader(
            dev_data,
            batch_size=BATCH_SIZE,
            shuffle=True,
            collate_fn=eval_wrapper,
            pin_memory=True,
            num_workers=10,
        )

        # reset model weights with each new fold
        model = BiLSTM(**params).to(device)
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        # normal training loop
        for epoch in range(1, NUM_EPOCHS + 1):
            model.train()
            train_loss, train_acc, train_auc = process(
                model, train_loader, criterion, optimizer
            )

            model.eval()
            with torch.no_grad():
                val_loss, val_acc, val_auc = process(model, dev_loader, criterion)

            # save metrics
            save_metrics(
                windowsize,
                fold,
                epoch,
                train_loss,
                train_acc,
                train_auc,
                val_loss,
                val_acc,
                val_auc,
                path=logging_path,
            )

            print(
                f"Training:   [Size {windowsize}, Fold {fold:2d}, Epoch {epoch:2d}, Loss: {train_loss:8.6f}, Acc: {train_acc:.4f}, AUC: {train_auc:.4f}]"
            )
            print(f"Evaluation: [Size {windowsize}, Fold {fold:2d}, Epoch {epoch:2d}, Loss: {val_loss:8.6f}, Acc: {val_acc:.4f}, AUC: {val_auc:.4f}]")

            reg_auc = regularized_auc(train_auc, val_auc, threshold=0)
            if reg_auc > highest_val_auc:
                highest_val_auc = reg_auc
                _path = path + f"size{windowsize}_auc{reg_auc:.4f}_fold{fold}_epoch{epoch}.pt"
                torch.save(model.state_dict(), _path)

print("Finished Cross-Validation.")
train_time = (time() - start) / 60
print(f"Cross-Validation took {train_time} minutes.")